<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD-GA Analysis mode Tutorial

FAST-OAD-GA is an add-on package for [FAST-OAD framework](https://github.com/fast-aircraft-design/FAST-OAD) for performing rapid Overall Aircraft Design in the category General Aviation (GA). The computational core of FAST-OAD being based on the  [OpenMDAO framework](https://openmdao.org/).

This notebook will show you the basic step to use the load analysis module using ID's to call the module instead of importing the classes.

To organize our work, we propose to use two user folders `data/` and `workdir/`. In `data/` we store the XML file which describes the aircraft to analyse, here the Beechcraft Duchess. In `workdir/`, we store files generated or modified by FAST-OAD-(GA).

In [ ]:
import os.path as pth
import os
import openmdao.api as om
from fastoad import api as api_cs25
from fastga.command import api as api_cs23
import logging
from fastoad.gui import VariableViewer
import shutil

# Define relative path
DATA_FOLDER_PATH = "data"
WORK_FOLDER_PATH = "workdir"

# Final file names
AIRCRAFT1_FILE = pth.join(WORK_FOLDER_PATH, "beechcraft_76_loads.xml")

In [ ]:
# Clear work folder
shutil.rmtree(WORK_FOLDER_PATH, ignore_errors=True)
os.mkdir(WORK_FOLDER_PATH)

# For using all screen width
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:95% !important; }</style>"))

## 1. Read, modify and save different aircraft geometry configurations

In this paragraph we will first run the geometry module on the aircraft to derive primary parameters into the detailed geometry set.

**This process can be done using the api generate block analysis method to get this working such as a python function:**
1. Create a dictionnary with the options for your module
2. Generate a block analysis using the module id

In [ ]:
# Copy reference aircraft file in work directory
shutil.copy(pth.join(DATA_FOLDER_PATH, "beechcraft_76_loads.xml"), AIRCRAFT1_FILE)

# Define the wing primary geometry parameters name as a list
var_inputs = [
    "data:geometry:wing:area",
    "data:geometry:wing:aspect_ratio",
    "data:geometry:wing:taper_ratio",
]

# Define the option of the geometry module you want to use (not putting any will use the default value when
# available)
option_dict = {"propulsion_id": "fastga.wrapper.propulsion.basicIC_engine"}

# Declare function
compute_geometry = api_cs23.generate_block_analysis(
    "fastga.geometry.legacy",
    var_inputs,
    str(AIRCRAFT1_FILE),
    options=option_dict,
    overwrite=True,
)

Now we can use the previously created function to calculate the aircraft.

In [ ]:
# Compute geometry
inputs_dict = {
    "data:geometry:wing:area": (16.638, "m**2"),
    "data:geometry:wing:aspect_ratio": (7.973, None),
    "data:geometry:wing:taper_ratio": (1.0, None),
}
outputs_dict = compute_geometry(inputs_dict)

# Open viewer
api_cs25.variable_viewer(AIRCRAFT1_FILE)

We will finish by visualizing the previously generated geometries before performing other calculation/analysis steps.

In [ ]:
from fastga.utils.postprocessing.analysis_and_plots import aircraft_geometry_plot

fig = aircraft_geometry_plot(
    pth.join(WORK_FOLDER_PATH, "beechcraft_76_loads.xml"), name="Beechcraft 76"
)
fig.show()

## 2. Performing aerodynamic analysis
In this chapter we will see how to perform an aerodynamics analysis on the aircraft.

To do so, we will launch the aerodynamic coefficients computation using the same technique as previously mentionned on geometry.

In [ ]:
# Define the option of the geometry module you want to use
option_dict = {
    "propulsion_id": "fastga.wrapper.propulsion.basicIC_engine",
    "use_openvsp": True,
    "compute_mach_interpolation": True,
    "compute_slipstream_cruise": True,
}

# Declare function on 1st geometry file
compute_aero1 = api_cs23.generate_block_analysis(
    "fastga.aerodynamics.legacy",
    [],
    str(AIRCRAFT1_FILE),
    options=option_dict,
    overwrite=True,
)

In [ ]:
# Compute both aircraft designs
result1 = compute_aero1({})

# Open viewer
api_cs25.variable_viewer(AIRCRAFT1_FILE)

In [ ]:
# Here we can verify the genral shape of the  lift repartition on the wing and the values found in the mach interpolation
import plotly
import plotly.graph_objects as go
import numpy as np
from fastoad.io import VariableIO
from fastga.models.load_analysis.wing.aerostructural_loads import AerostructuralLoad

COLS = plotly.colors.DEFAULT_PLOTLY_COLORS

variables = VariableIO(AIRCRAFT1_FILE, None).read()

y_vector = np.array(list(variables["data:aerodynamics:wing:low_speed:Y_vector"].value))
CL_vector = np.array(
    list(variables["data:aerodynamics:slipstream:wing:cruise:prop_on:CL_vector"].value)
)

y_vector = AerostructuralLoad.delete_additional_zeros(y_vector)
CL_vector = AerostructuralLoad.delete_additional_zeros(CL_vector)

span = variables["data:geometry:wing:span"].value
semi_span = span[0] / 2.0

if abs(CL_vector[-1]) > 0.01:
    y_vector = np.append(y_vector, semi_span)
    CL_vector = np.append(CL_vector, 0.0)

fig = go.Figure()

lift_repartition = go.Scatter(x=y_vector, y=CL_vector, mode="lines", name="Beechcraft 76")
fig.add_trace(lift_repartition)
fig.update_layout(
    title_text="Lift coefficient repartition on the wing",
    title_x=0.5,
    xaxis_title="Position along the span [m]",
    yaxis_title="Lift coefficient [-]",
)
fig.show()

In [ ]:
mach_vector = list(variables["data:aerodynamics:aircraft:mach_interpolation:mach_vector"].value)
Cl_alpha_vector = list(
    variables["data:aerodynamics:aircraft:mach_interpolation:CL_alpha_vector"].value
)

fig2 = go.Figure()

mach_interpolation = go.Scatter(
    x=mach_vector, y=Cl_alpha_vector, mode="lines", name="Beechcraft 76"
)
fig2.add_trace(mach_interpolation)
fig2.update_layout(
    title_text="Lift coefficient slope as a function of Mach number",
    title_x=0.5,
    xaxis_title="Mach number [-]",
    yaxis_title="Lift coefficient slope [-]",
)
fig2.show()

Then, we will compute the most stringent loading case among the one we defined in the function (MTOW and Min fuel weight in the wing) so that we can then print out the shear stress and wing root bending moment. First we must compute the maximum level velocity at sea level as it is an input of the V-n diagrams computed during the wing loading computation process. Since this component alone is not registered as a module, we can't generate the block analysis using only an ID but we can use the component directly

In [ ]:
from fastga.models.aerodynamics.components.compute_vn import ComputeVh

var_inputs = ["data:weight:aircraft:MTOW"]

compute_geometry1 = api_cs23.generate_block_analysis(
    ComputeVh(propulsion_id="fastga.wrapper.propulsion.basicIC_engine"),
    var_inputs,
    str(AIRCRAFT1_FILE),
    overwrite=True,
)

In [ ]:
inputs_dict = {"data:weight:aircraft:MTOW": (1769.0, "kg")}

result1 = compute_geometry1(inputs_dict)

# Open viewer
api_cs25.variable_viewer(AIRCRAFT1_FILE)

For the loads however, we can go back to using ID's

In [ ]:
var_inputs = [
    "data:weight:aircraft:MZFW",
    "data:weight:aircraft:CG:fwd:x",
    "data:weight:aircraft:CG:aft:x",
    "data:weight:aircraft_empty:CG:z",
    "data:weight:propulsion:engine:mass",
    "data:weight:airframe:landing_gear:main:mass",
    "data:weight:airframe:wing:mass",
    "data:mission:sizing:fuel",
]

compute_geometry1 = api_cs23.generate_block_analysis(
    "fastga.loads.wing",
    var_inputs,
    str(AIRCRAFT1_FILE),
    overwrite=True,
)

In [ ]:
# Compute both aircraft designs using similar input parameters
inputs_dict = {
    "data:weight:aircraft:MZFW": (1531.6, "kg"),
    "data:weight:aircraft:CG:fwd:x": (2.76, "m"),
    "data:weight:aircraft:CG:aft:x": (3.09, "m"),
    "data:weight:aircraft_empty:CG:z": (1.197, "m"),
    "data:weight:propulsion:engine:mass": (351.84, "kg"),
    "data:weight:airframe:landing_gear:main:mass": (55.34, "kg"),
    "data:weight:airframe:wing:mass": (192.46, "kg"),
    "data:mission:sizing:fuel": (150.0, "kg"),
}

result1 = compute_geometry1(inputs_dict)
# Open viewer
api_cs25.variable_viewer(AIRCRAFT1_FILE)

In [ ]:
from fastga.utils.postprocessing.load_analysis.analysis_and_plots_la import (
    force_repartition_diagram,
)

fig = force_repartition_diagram(
    pth.join(WORK_FOLDER_PATH, "beechcraft_76_loads.xml"), name="Beechcraft Duchess"
)
fig.show()

In [ ]:
from fastga.utils.postprocessing.load_analysis.analysis_and_plots_la import shear_diagram

fig2 = shear_diagram(
    pth.join(WORK_FOLDER_PATH, "beechcraft_76_loads.xml"), name="Beechcraft Duchess"
)
fig2.show()

In [ ]:
from fastga.utils.postprocessing.load_analysis.analysis_and_plots_la import rbm_diagram

fig3 = rbm_diagram(pth.join(WORK_FOLDER_PATH, "beechcraft_76_loads.xml"), name="Beechcraft Duchess")
fig3.show()